In [ ]:
!pip install trl -U

In [1]:
# imports

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

import numpy as np
import torch

In [2]:
import trl

In [3]:
 # SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

In [4]:
# despite the name, this contains both train and val, which you can use split to distinguish
preference = load_dataset("nvidia/HelpSteer2", data_dir="preference")['train'] 

In [5]:
preference

Dataset({
    features: ['split', 'prompt', 'response_1', 'response_2', 'preference_strength', 'preference_statement', 'preference_elaboration', 'three_most_similar_preferences', 'all_preferences_unprocessed'],
    num_rows: 9125
})

In [6]:
preference['prompt'][0:5]

['Define Signal Discuss its various properties with the help of diagram',
 'Please make a list of independent Fertility coaching and consulting services in USA\n\n',
 'Explain various data mining functionalities with examples?',
 'How do I change the short cut on mac osx to toggle the voiceover feature?',
 'I have a project where we deploy the "Vertex AI Vision SDK" into a container in Docker Desktop on the client computer on the edge and the sdk pushes the camera feed from the rtsp to Google Cloud\'s Vertex AI Vision and it goes through pre-trained human/vehicle model to find out count and that information is written into BigQuery in json format. This information will be queried and provided back to the customer in json response. \n\nI would like to identify the roundtrip time from the start of the stream to the json response. However the timestamp on the camera is off. It\'s hard to evaluate the roundtrip time. \n\nProvide me methodologies/ideas on how to accurately calculate the rou

In [7]:
np.unique(np.array(preference['preference_strength']), return_counts=True)

(array([-3, -2, -1,  0,  1,  2,  3]),
 array([ 751, 1234, 1346, 2007, 1687, 1328,  772]))

In [8]:
np.unique(np.array(preference['split']), return_counts=True)

(array(['train', 'val'], dtype='<U5'), array([8677,  448]))

In [9]:
# if strength is in [-3, 0] choose the first,
# otherwise the second

In [10]:
train = []
val = []
for x in preference:
    prompt = x['prompt']
    if x['preference_strength'] <= 0:
        response = x['response_1']
    else: 
        response = x['response_2']
        
    if x['split'] == 'train':
        train.append({'question':prompt, 'answer': response})
    else:
        val.append({'question':prompt, 'answer': response})

In [11]:
len(train), len(val)

(8677, 448)

In [12]:
train[0]

{'question': 'Define Signal Discuss its various properties with the help of diagram',
 'answer': 'A signal is a message that is conveyed from a sender to a receiver through a communication channel. The message can be in the form of a sound, light, electrical, or any other form that can be transmitted and received. The properties of a signal can vary depending on the type of signal and the communication channel used. Here are some common properties of signals:\n\nAmplitude: This is the maximum value that a signal can take. It represents the strength of the signal. For example, in an electrical signal, amplitude can be measured in volts.\n\nFrequency: This is the number of cycles or oscillations of a signal per unit time. It is measured in hertz (Hz). For example, a sound signal with a frequency of 20 Hz vibrates 20 times per second.\n\nWavelength: This is the distance between two successive crests or troughs of a signal. It is measured in meters (m). For example, in a light signal, wave

In [13]:
from datasets import Dataset

In [14]:
train_dataset = Dataset.from_list(train)
val_dataset = Dataset.from_list(val)

In [15]:
train_dataset.save_to_disk("train_sft_HelpSteer2")
val_dataset.save_to_disk("val_sft_HelpSteer2")

Saving the dataset (0/1 shards):   0%|          | 0/8677 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/448 [00:00<?, ? examples/s]

In [16]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

In [17]:
model=AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

In [18]:
model.to('cuda')

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features

In [19]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [21]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system names
        if name == "PIL":
            name = "Pillow"
        elif name == "sklearn":
            name = "scikit-learn"

        yield name
imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

/tmp/ipykernel_474/758457530.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


datasets==3.1.0
numpy==1.26.4
torch==2.4.0
transformers==4.46.3
trl==0.12.1


In [22]:
# одну эпоху на chosen примерах из датасета. Используйте learning rate = 1e-4.

training_args = trl.SFTConfig(learning_rate=1e-4, fp16=True, num_train_epochs=1, save_strategy="no", save_total_limit=1, logging_steps=50, output_dir="/kaggle/working/")

response_template = " ### Answer:"
collator = trl.DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = trl.SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args,
    max_seq_length=512,
    formatting_func=formatting_prompts_func,
    data_collator=collator
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/8677 [00:00<?, ? examples/s]

Map:   0%|          | 0/448 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key ` ### Answer:` in the following instance: </s>### Question: you are a copywriter for my vegan community called "Vegan Foodie Tribe", its a monthly membership, do you understand:"?
<extra_id_1>Assistant
Yes, I am here to help you with the copywriting for your vegan community "Vegan Foodie Tribe". I understand that you want to create a monthly membership program to provide information and support for vegan foodies, and I am happy to help you with this. I will use my expertise in writing and marketing to create compelling copy for your membership program, and I look forward to working with you!
<extra_id_1>User
What should we include in the membership for $49
<extra_id_1>Assistant
For $49 a month, members of the Vegan Foodie Tribe will receive a digital membership card, access to an exclusive members-only Facebook group, monthly vegan recipe e-book, a monthly newsletter with articles

Step,Training Loss
50,2.007100
100,1.980400
150,2.006400
200,1.981500
250,2.009600
300,1.970100
350,1.931300
400,1.927300
450,1.889100
500,1.919500


/opt/conda/lib/python3.10/site-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key ` ### Answer:` in the following instance: </s>### Question: You know how people like to watch cat videos online? Let's say there are two personified cats lying in bed watching videos of people (the same way people watch videos of cats). What would be some funny things they might say to each other? We could use some puns that have to do with things about cats or fish.
<extra_id_1>Assistant
Here are a few ideas:
 

 1. "Meow meow! Check out this video of a couple of humans playing with kitty litter!"
 

 2. "Pssst! This video is kind of a paw-dunk-a-delicious!"
 

 3. "Wow, these people really know how to fish!"
 

 4. "MEOW! Those humans are crazy! They're dancing to the fish!?"
 

 5. "Pssst! This video is pretty funny!"
<extra_id_1>User
What if the video they are watching has the humans smashing kitty litter with tools?
<extra_id_1>Assistant
Then I think they might say something 

TrainOutput(global_step=1085, training_loss=1.870099369172127, metrics={'train_runtime': 1629.858, 'train_samples_per_second': 5.324, 'train_steps_per_second': 0.666, 'total_flos': 8076024520507392.0, 'train_loss': 1.870099369172127, 'epoch': 1.0})

In [23]:
trainer.save_model("/kaggle/working//sft_opt")